In [19]:
import requests
from bs4 import BeautifulSoup
import sqlite3
import datetime
PARARIUS_URL = "https://www.pararius.com/apartments/rotterdam/0-1400/2-bedrooms"


In [20]:


# URL to check

def create_database(db_name="listings.db"):
    """Creates/connects to a SQLite database and creates the listings table if it doesn't exist."""
    conn = sqlite3.connect(db_name)
    c = conn.cursor()
    c.execute('''
        CREATE TABLE IF NOT EXISTS listings (
            id TEXT PRIMARY KEY,
            title TEXT,
            price TEXT,
            location TEXT,
            url TEXT,
            real_estate TEXT,
            date_added TEXT
        )
    ''')
    conn.commit()
    return conn

def fetch_listings():
    """Fetches the Pararius page and returns a list of listings with their details."""
    headers = {
        'User-Agent': ('Mozilla/5.0 (Windows NT 10.0; Win64; x64) '
                       'AppleWebKit/537.36 (KHTML, like Gecko) '
                       'Chrome/90.0.4430.93 Safari/537.36')
    }
    try:
        response = requests.get(PARARIUS_URL, headers=headers)
        response.raise_for_status()
    except requests.RequestException as e:
        print("Error fetching the page:", e)
        return []

    soup = BeautifulSoup(response.content, "html.parser")
    
    # Find listing containers; adjust these selectors to match the actual page structure.
    # Here we assume each listing is in a section with a class like "listing-search-item".
    listings = []
    for item in soup.find_all("section", class_="listing-search-item"):
        # Try to get a unique id for the listing.
        listing_id = item.get("data-id")
        if not listing_id:
            # As a fallback, use the detail page URL
            link = item.find("a", href=True)
            listing_id = link["href"] if link else None
        if not listing_id:
            continue  # Skip if no identifier is found

        # Extract other details (update selectors if needed)
        title_elem = item.find("h2", class_="listing-search-item__title")
        title = title_elem.get_text(strip=True) if title_elem else "No Title"
        
        price_elem = item.find("div", class_="listing-search-item__price")
        price = price_elem.get_text(strip=True) if price_elem else "No Price"
        
        location_elem = item.find("div", class_="listing-search-item__sub-title'")
        location = location_elem.get_text(strip=True) if location_elem else "No Location"

        real_estate_elem = item.find("div", class_="listing-search-item__info")
        if real_estate_elem:
            link_elem = real_estate_elem.find("a", class_="listing-search-item__link")
            if link_elem:
                real_estate = link_elem.get_text(strip=True)
            else:
                real_estate = "None"
        print(real_estate)
        
        link_elem = item.find("a", href=True)
        # Ensure the URL is absolute
        detail_url = ("https://www.pararius.com" + link_elem["href"]) if link_elem and link_elem["href"].startswith("/") else (link_elem["href"] if link_elem else "")

        listings.append({
            "id": listing_id,
            "title": title,
            "price": price,
            "location": location,
            "url": detail_url,
            "real_estate": real_estate,
            "date_added": datetime.datetime.now().isoformat(),
        })
    return listings

def check_new_listings(conn, listings):
    """Compares fetched listings with the local database and returns new listings."""
    c = conn.cursor()
    new_listings = []
    for listing in listings:
        c.execute("SELECT id FROM listings WHERE id = ?", (listing["id"],))
        result = c.fetchone()
        if result is None:
            # This is a new listing
            new_listings.append(listing)
            c.execute(
                "INSERT INTO listings (id, title, price, location, url, date_added, real_estate) VALUES (?, ?, ?, ?, ?, ?, ?)",
                (listing["id"], listing["title"], listing["price"], listing["location"], listing["url"], listing["date_added"], listing["real_estate"])
            )
            conn.commit()
    return new_listings

def main():
    conn = create_database()
    listings = fetch_listings()
    if not listings:
        print("No listings found or error in fetching the page.")
        return

    new_listings = check_new_listings(conn, listings)
    if new_listings:
        print("New listings found:")
        for listing in new_listings:
            print(f"- {listing['title']} | {listing['price']} | {listing['location']}")
            print(f"  URL: {listing['url']}\n")
    else:
        print("No new listings found.")
    
    conn.close()

if __name__ == "__main__":
    main()


realestate.nl
Rental Rotterdam
Expat Housing Rotterdam
Huizenbalie.nl
VERRA Makelaars Rotterdam
realestate.nl
D&S Vastgoed
realestate.nl
Kolpa V.V.S. Beheer
Rent an Apartment
Rent an Apartment
Properbee
Properbee
Properbee
New listings found:
- Flat Grieksestraat | €985 per month | 3028 CP Rotterdam (Oud Mathenesse)
  URL: https://www.pararius.com/apartment-for-rent/rotterdam/f4600238/grieksestraat

- Flat Dordtselaan | €924 per month | 3073 GW Rotterdam (Bloemhof)
  URL: https://www.pararius.com/apartment-for-rent/rotterdam/b7e665fb/dordtselaan

- Flat Abeelweg | €1,350 per month | 3053 PD Rotterdam (Schiebroek)
  URL: https://www.pararius.com/apartment-for-rent/rotterdam/86c8d1aa/abeelweg

- Flat Aristotelesstraat 93 | €1,250 per month | 3076 BC Rotterdam (Lombardijen)
  URL: https://www.pararius.com/apartment-for-rent/rotterdam/cfe22770/aristotelesstraat

- Flat Lange Hilleweg | €1,092 per month | 3073 BZ Rotterdam (Bloemhof)
  URL: https://www.pararius.com/apartment-for-rent/rotter

3028 CP Rotterdam (Oud Mathenesse)
3073 GW Rotterdam (Bloemhof)
3053 PD Rotterdam (Schiebroek)
3076 BC Rotterdam (Lombardijen)
3073 BZ Rotterdam (Bloemhof)
3032 RE Rotterdam (Agniesebuurt)
3083 CR Rotterdam (Zuidplein)
3076 DK Rotterdam (Lombardijen)
3082 SB Rotterdam (Oud Charlois)
3085 EN Rotterdam (Zuidwijk)
3061 MC Rotterdam (Kralingen West)
3054 NS Rotterdam (Hillegersberg Noord)
3082 LC Rotterdam (Oud Charlois)
3073 HC Rotterdam (Bloemhof)
